In [8]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer



In [9]:
database=pd.read_csv("database_corregido.csv")

In [10]:
database.head(15)

,user_id,user_name,usersurname,email,password,year_bird,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
0,1,ZpafJwb,IGLxqLcD,SidQXyFROW@gmail.com,FuDgmKsOA,1997,Femenino,"Frances, Chino",De 1 año a 2 años,Sin graduado escolar,Si,Emocional,No,Valle del Jarama,"Juegos, Baile, Musica",2021-10-13,NaN,NaN
1,2,vraVenl,pQgDlKpb,dmRMlEHSix@gmail.com,TcEDUaWhT,1983,No especificar,"Aleman, Chino",De 1 año a 2 años,Master,No especificar,Orientacion sobre temas legales,Si,Valle Bajo del Lozoya,"Lectura, Baile, Cine",2021-07-12,NaN,NaN
2,3,ZCogidr,pyqWoZEy,InOMyDalbC@gmail.com,YigDCDzGH,1998,Masculino,"Portugues, Ingles",Menos de 6 meses,Doctorado,No especificar,Orientacion laboral,No,Sierra del Rincon,"Viajar, Baile, Lectura",2021-02-09,NaN,NaN
3,4,vdTiFuY,DsyOuIJT,zXBFBgHyDk@gmail.com,juZCtMEMk,1968,Femenino,"Frances, Ucraniano",De 6 meses a 1 año,FP (Formacion Profesional),No especificar,Orientación sobre trámites,Si,Valle Medio del Lozoya,"Deportes, Pintura, Baile",2022-08-24,NaN,NaN
4,5,sRnfLQq,sOAsLqnv,juObUXPbPT@gmail.com,IlpRnxgfA,1982,No especificar,"Ucraniano, Ingles",De 6 meses a 1 año,Sin graduado escolar,Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Cocina, Juegos, Musica",2021-04-21,NaN,NaN
5,6,mzMaFwG,aFluzzgb,TLUFIGikzn@gmail.com,pygSbGQYX,1980,Masculino,"Chino, Aleman",Más de 2 años,Master,No especificar,Orientacion sobre temas legales,No,Valle Medio del Lozoya,"Baile, Naturaleza, Musica",2021-02-26,NaN,NaN
6,7,YfbbBZK,oVJPFoau,EjdaYjbpAb@gmail.com,oiFAMfhta,1997,Femenino,"Italiano, Chino",De 6 meses a 1 año,Sin graduado escolar,No,Orientacion laboral,No,Valle Medio del Lozoya,"Musica, Baile, Naturaleza",2022-04-13,NaN,NaN
7,8,qUfSMUQ,ZiBYiGIV,IaaQlRmgjO@gmail.com,ZYsipCHZe,1966,No especificar,Español,Menos de 6 meses,FP (Formacion Profesional),Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Pintura, Deportes, Fotografia",2022-09-14,NaN,NaN
8,9,OZCBNeY,tznFIrsc,OjWwToudCa@gmail.com,rWdpKbTYS,1966,No especificar,Frances,Menos de 6 meses,Master,No especificar,Emocional,Si,Valle Bajo del Lozoya,"Musica, Fotografia, Cocina",2021-07-14,NaN,NaN
9,10,hnTeOkv,PlxNjQhC,nHiblclrAl@gmail.com,lkyfissrp,1973,No especificar,Aleman,Menos de 6 meses,Doctorado,No,Orientación sobre trámites,Si,Sierra del Rincon,"Baile, Musica, Juegos",2022-05-13,NaN,NaN


In [11]:
df = database.copy()


### MODIFICO EL AÑO DE NACIMIENTO PARA PODER INCLUIRLO COMO RANGO DE EDAD

In [12]:
labels=['De55a65', 'De45a55', 'De35a45', 'De25a35', 'De18a25']
df['year_bird'] = pd.cut(df['year_bird'], bins=[1957, 1967, 1977, 1987, 1997, 2005], labels=labels)
df['year_bird'] = df['year_bird'].astype(str)
df['year_bird'].value_counts()


De25a35    252
De35a45    243
De45a55    240
De18a25    162
De55a65    103
Name: year_bird, dtype: int64

#### JUNTO LAS PALABRAS DE LAS COLUMNAS COMARCAS Y NIVEL DE ESTUDIOS PARA QUE SEAN UNA SOLA PALABRA

In [13]:
df['area'] = df['area'].apply(lambda x: re.sub(' ', '', x))
df['studies'] = df['studies'].apply(lambda x: re.sub(' ', '', x))
print(df['area'])
print(df['studies'])

0           ValledelJarama
1       ValleBajodelLozoya
2          SierradelRincon
3      ValleMediodelLozoya
4       ValleAltodelLozoya
              ...         
995        SierradelRincon
996         ValledelJarama
997        SierradelRincon
998    ValleMediodelLozoya
999         ValledelJarama
Name: area, Length: 1000, dtype: object
0            Singraduadoescolar
1                        Master
2                     Doctorado
3      FP(FormacionProfesional)
4            Singraduadoescolar
                 ...           
995          Gradouniversitario
996          Gradouniversitario
997          Gradouniversitario
998          Singraduadoescolar
999                   Doctorado
Name: studies, Length: 1000, dtype: object


In [14]:
# Unir varias columnas en una sola

df['all_about_me'] = df['about_me'] + ', ' + df['mother_tongue'] + ', ' + df['area'] + ', ' + df['studies'] + ', ' + df['year_bird']
df['all_about_me']


0      Juegos, Baile, Musica, Frances, Chino, Vallede...
1      Lectura, Baile, Cine, Aleman, Chino, ValleBajo...
2      Viajar, Baile, Lectura, Portugues, Ingles, Sie...
3      Deportes, Pintura, Baile, Frances, Ucraniano, ...
4      Cocina, Juegos, Musica, Ucraniano, Ingles, Val...
                             ...                        
995    Pintura, Cine, Naturaleza, Aleman, SierradelRi...
996    Juegos, Fotografia, Baile, Español, ValledelJa...
997    Musica, Pintura, Cocina, Aleman, SierradelRinc...
998    Baile, Juegos, Viajar, Español, Aleman, ValleM...
999    Cocina, Baile, Fotografia, Rumano, Arabe, Vall...
Name: all_about_me, Length: 1000, dtype: object

In [15]:
df['all_about_me'] = df['all_about_me'].apply(lambda x: nltk.word_tokenize(x))
df['all_about_me']

0      [Juegos, ,, Baile, ,, Musica, ,, Frances, ,, C...
1      [Lectura, ,, Baile, ,, Cine, ,, Aleman, ,, Chi...
2      [Viajar, ,, Baile, ,, Lectura, ,, Portugues, ,...
3      [Deportes, ,, Pintura, ,, Baile, ,, Frances, ,...
4      [Cocina, ,, Juegos, ,, Musica, ,, Ucraniano, ,...
                             ...                        
995    [Pintura, ,, Cine, ,, Naturaleza, ,, Aleman, ,...
996    [Juegos, ,, Fotografia, ,, Baile, ,, Español, ...
997    [Musica, ,, Pintura, ,, Cocina, ,, Aleman, ,, ...
998    [Baile, ,, Juegos, ,, Viajar, ,, Español, ,, A...
999    [Cocina, ,, Baile, ,, Fotografia, ,, Rumano, ,...
Name: all_about_me, Length: 1000, dtype: object

In [16]:
def removeStopwords(text):
	"""Se eliminan palabras que carecen de significado
    
    Args: 
        text: Texto del que se desea eliminar las palabras sin significado.
        
    """
	
	stopw = nltk.corpus.stopwords.words('spanish')
	x = [w.strip() for w in text if w not in stopw]
	return x

In [17]:
df['all_about_me'] = df['all_about_me'].apply(lambda x: removeStopwords(x))
df['all_about_me']

0      [Juegos, ,, Baile, ,, Musica, ,, Frances, ,, C...
1      [Lectura, ,, Baile, ,, Cine, ,, Aleman, ,, Chi...
2      [Viajar, ,, Baile, ,, Lectura, ,, Portugues, ,...
3      [Deportes, ,, Pintura, ,, Baile, ,, Frances, ,...
4      [Cocina, ,, Juegos, ,, Musica, ,, Ucraniano, ,...
                             ...                        
995    [Pintura, ,, Cine, ,, Naturaleza, ,, Aleman, ,...
996    [Juegos, ,, Fotografia, ,, Baile, ,, Español, ...
997    [Musica, ,, Pintura, ,, Cocina, ,, Aleman, ,, ...
998    [Baile, ,, Juegos, ,, Viajar, ,, Español, ,, A...
999    [Cocina, ,, Baile, ,, Fotografia, ,, Rumano, ,...
Name: all_about_me, Length: 1000, dtype: object

In [18]:
def removePunctuation(text):
	"""Se eliminan signos de puntuación
    
    Args: 
        text: Texto del que se desea eliminar los sigbos de puntuación.
        
    """
	stopp = list(string.punctuation)
	stopp.append("''")
	stopp.append("")
	x = [w.strip() for w in text if w not in stopp]
	return x

In [19]:
df['all_about_me'] = df['all_about_me'].apply(lambda x: removePunctuation(x))
df['all_about_me']

0      [Juegos, Baile, Musica, Frances, Chino, Valled...
1      [Lectura, Baile, Cine, Aleman, Chino, ValleBaj...
2      [Viajar, Baile, Lectura, Portugues, Ingles, Si...
3      [Deportes, Pintura, Baile, Frances, Ucraniano,...
4      [Cocina, Juegos, Musica, Ucraniano, Ingles, Va...
                             ...                        
995    [Pintura, Cine, Naturaleza, Aleman, SierradelR...
996    [Juegos, Fotografia, Baile, Español, ValledelJ...
997    [Musica, Pintura, Cocina, Aleman, SierradelRin...
998    [Baile, Juegos, Viajar, Español, Aleman, Valle...
999    [Cocina, Baile, Fotografia, Rumano, Arabe, Val...
Name: all_about_me, Length: 1000, dtype: object

In [20]:
def arrayToString(text):
	"""Se eliminan signos de puntuación
    
    Args: 
        text: Texto del que se desea eliminar los sigbos de puntuación.
        
    """
	x = ' '.join(text)
	return x

In [21]:
df['all_about_me'] = df['all_about_me'].apply(lambda x: arrayToString(x))
df['all_about_me']

0      Juegos Baile Musica Frances Chino ValledelJara...
1      Lectura Baile Cine Aleman Chino ValleBajodelLo...
2      Viajar Baile Lectura Portugues Ingles Sierrade...
3      Deportes Pintura Baile Frances Ucraniano Valle...
4      Cocina Juegos Musica Ucraniano Ingles ValleAlt...
                             ...                        
995    Pintura Cine Naturaleza Aleman SierradelRincon...
996    Juegos Fotografia Baile Español ValledelJarama...
997    Musica Pintura Cocina Aleman SierradelRincon G...
998    Baile Juegos Viajar Español Aleman ValleMediod...
999    Cocina Baile Fotografia Rumano Arabe ValledelJ...
Name: all_about_me, Length: 1000, dtype: object

## Listado de opciones para recomendar a los usuarios

In [23]:
Listado = df['user_id']
Listado

0         1
1         2
2         3
3         4
4         5
       ... 
995     996
996     997
997     998
998     999
999    1000
Name: user_id, Length: 1000, dtype: int64

##### Cálculo Matriz Frecuencias (CountVectorizer()

In [25]:
vectorizer = CountVectorizer()
MatrizFrecuencias = vectorizer.fit_transform(df['all_about_me'])
MatrizFrecuencias

<1000x39 sparse matrix of type '<class 'numpy.int64'>'
	with 7672 stored elements in Compressed Sparse Row format>

##### Visualización Matriz Frecuencias

In [26]:
X = pd.DataFrame(data=MatrizFrecuencias.toarray(), index=Listado.values,
                columns=vectorizer.get_feature_names())
X

c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aleman,arabe,baile,chino,cine,cocina,congraduadoescolar,de18a25,de25a35,de35a45,...,rumano,sierradelacabrera,sierradelrincon,singraduadoescolar,ucraniano,vallealtodellozoya,vallebajodellozoya,valledeljarama,vallemediodellozoya,viajar
1,0,0,1,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
2,1,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
5,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
997,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
998,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
999,1,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1


##### Cálculo término frecuencia inversa de documento (TfidfTransformer())

In [27]:
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(MatrizFrecuencias)
tfidf

<1000x39 sparse matrix of type '<class 'numpy.float64'>'
	with 7672 stored elements in Compressed Sparse Row format>

##### Visualizacion Matriz con términos frecuencia inversa de documento y normalizada

In [28]:
X_norm = pd.DataFrame(data = tfidf.toarray(),index=Listado.values,
                     columns=vectorizer.get_feature_names())
X_norm

c:\Users\Alex\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aleman,arabe,baile,chino,cine,cocina,congraduadoescolar,de18a25,de25a35,de35a45,...,rumano,sierradelacabrera,sierradelrincon,singraduadoescolar,ucraniano,vallealtodellozoya,vallebajodellozoya,valledeljarama,vallemediodellozoya,viajar
1,0.000000,0.000000,0.297901,0.396501,0.000000,0.000000,0.0,0.000000,0.327217,0.000000,...,0.000000,0.0,0.000000,0.382800,0.000000,0.000000,0.000000,0.379577,0.000000,0.000000
2,0.395203,0.000000,0.294218,0.391599,0.318416,0.000000,0.0,0.000000,0.000000,0.328100,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.392491,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.291353,0.000000,0.000000,0.000000,0.0,0.379256,0.000000,0.000000,...,0.000000,0.0,0.362199,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.304928
4,0.000000,0.000000,0.275295,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.368088,0.000000,0.000000,0.000000,0.356031,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.314666,0.0,0.000000,0.000000,0.326876,...,0.000000,0.0,0.000000,0.376657,0.391922,0.381557,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.432558,0.000000,0.000000,0.000000,0.348513,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.400332,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
997,0.000000,0.000000,0.325518,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.414766,0.000000,0.000000
998,0.433715,0.000000,0.000000,0.000000,0.000000,0.346622,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.401403,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
999,0.400083,0.000000,0.297851,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.382735,0.000000,0.000000,0.000000,0.000000,0.385202,0.311728


##### Cálculo matriz similitudes

In [29]:
tdm = tfidf.transpose()
dtm = tfidf
Simil = dtm.dot(tdm)

##### Visualizacion Matriz Similitudes

In [30]:
SimilDF = pd.DataFrame(data = Simil.toarray(), index=Listado.values,columns=Listado.values)
SimilDF

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
1,1.000000,0.242917,0.086794,0.227956,0.339427,0.343000,0.597400,0.000000,0.260985,0.311544,...,0.105848,0.090693,0.262140,0.302134,0.436258,0.000000,0.362990,0.107365,0.334595,0.335471
2,0.242917,1.000000,0.183246,0.080997,0.107248,0.650345,0.242422,0.000000,0.304756,0.266649,...,0.166672,0.338752,0.158826,0.098619,0.230172,0.281921,0.095773,0.171405,0.245747,0.086507
3,0.086794,0.183246,1.000000,0.080208,0.159998,0.086288,0.086617,0.000000,0.000000,0.402781,...,0.101589,0.189613,0.000000,0.258475,0.086200,0.145000,0.094841,0.145388,0.181834,0.234049
4,0.227956,0.080997,0.080208,1.000000,0.144262,0.217898,0.218731,0.443378,0.148216,0.200751,...,0.266767,0.323122,0.094532,0.135657,0.081449,0.211280,0.202201,0.211844,0.322159,0.080943
5,0.339427,0.107248,0.159998,0.144262,1.000000,0.204854,0.241158,0.140373,0.201178,0.211813,...,0.270961,0.000000,0.202635,0.354886,0.197047,0.000000,0.106840,0.214713,0.241919,0.099320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.000000,0.281921,0.145000,0.211280,0.000000,0.281632,0.109973,0.102559,0.000000,0.479375,...,0.128790,0.000000,0.110579,0.150864,0.000000,1.000000,0.298385,0.761446,0.293565,0.000000
997,0.362990,0.095773,0.094841,0.202201,0.106840,0.096406,0.096774,0.261982,0.115723,0.355173,...,0.251949,0.099101,0.000000,0.259886,0.360505,0.298385,1.000000,0.299182,0.489496,0.363563
998,0.107365,0.171405,0.145388,0.211844,0.214713,0.279276,0.107146,0.102833,0.221609,0.597134,...,0.243861,0.000000,0.334088,0.257451,0.110090,0.761446,0.299182,1.000000,0.294350,0.109406
999,0.334595,0.245747,0.181834,0.322159,0.241919,0.394909,0.382864,0.140796,0.000000,0.498666,...,0.222976,0.242342,0.000000,0.245032,0.186796,0.293565,0.489496,0.294350,1.000000,0.087575


##### Ejemplo Recomendación por Similitud entre items

In [31]:
Top = 10
ID_usuario = (10) -1 # Modificar sólo el dato entre paréntesis
print('Usuario:',SimilDF.columns[ID_usuario])
RecomendacionItemItem = SimilDF.iloc[(-SimilDF.iloc[:, ID_usuario]).argsort()[1:(Top+1)].values, ID_usuario]
print ('\n Los usuarios mas compatibles son:')
RecomendacionItemItem

Usuario: 10

 Los usuarios mas compatibles son:


92     0.696083
758    0.679715
705    0.645852
488    0.645496
763    0.636192
998    0.597134
872    0.596122
436    0.591990
183    0.588577
501    0.586811
Name: 10, dtype: float64

In [32]:
import json

reco = RecomendacionItemItem.to_json()
reco

'{"92":0.6960829651,"758":0.6797150032,"705":0.6458522185,"488":0.6454959684,"763":0.636191917,"998":0.5971341584,"872":0.5961215508,"436":0.5919902533,"183":0.5885769565,"501":0.5868110423}'